## Code to download selected tiles of MA lidar that intersect/overlap with Worcester City Limits

### Must be run in ArcPro

In [47]:
import arcpy
import requests
import os

In [48]:
import pathlib

In [49]:
arcpy.env.workspace = "G:\My Drive\Clark\HERO\HERO Geospatial"
lidar_cache = "G:\\My Drive\\Clark\\HERO\\HERO Geospatial\\WooLidar\\lidarcache\\"

In [50]:
lidar_cache

'G:\\My Drive\\Clark\\HERO\\HERO Geospatial\\WooLidar\\lidarcache\\'

In [51]:
address = "https://rockyweb.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/MA_CentralEastern_2021_B21/MA_CentralEastern_1_2021/LAZ/USGS_LPC_MA_CentralEastern_2021_B21_"
tiles = []
with arcpy.da.SearchCursor("WORCESTER_LIDAR_TILES", ["TILENAME"]) as cursor:
    for row in cursor:
        tiles.append(address+row[0]+".laz")

In [52]:
tiles[1]

'https://rockyweb.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/MA_CentralEastern_2021_B21/MA_CentralEastern_1_2021/LAZ/USGS_LPC_MA_CentralEastern_2021_B21_19TBG270690.laz'

In [53]:
def all_tiles_to_dataset(tiles):
    # The function will perform all actions for a tile then delete the source .laz to consesrve disk space
    for tile in tiles:
        # get tile from address
        response = requests.get(tile)
        filename = lidar_cache + tile[-14:]
        with open(filename, 'wb') as file:
            file.write(response.content)
        # convert laz to las
        arcpy.conversion.ConvertLas(filename, lidar_cache)
        # remove las
        os.remove(filename)
    # create las dataset from las files
    arcpy.management.CreateLasDataset(lidar_cache, "worcester.lasd", "RECURSION")


In [54]:
all_tiles_to_dataset(tiles)